In [1]:
from flask_cors import CORS
from flask import Flask, render_template, request, redirect,session, url_for,jsonify
from flask_sqlalchemy import SQLAlchemy

import os
from distutils.util import strtobool


In [2]:
app = Flask(__name__)
CORS(app)  # Enable CORS for all routes
app.secret_key = os.urandom(24)

app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///users.db'
db = SQLAlchemy(app)

In [3]:
class Student(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(80), unique=False, nullable=False)
    gender = db.Column(db.String(80), unique=False, nullable=False)
    dob = db.Column(db.String(80), unique=False, nullable=False)
    mobile = db.Column(db.String(120), unique=False, nullable=False)
    email = db.Column(db.String(120), unique=False, nullable=False)
    image = db.Column(db.String(100), nullable=False)
    parentMobile = db.Column(db.String(120), unique=False, nullable=False)
    courses = db.relationship('Course', backref='student', lazy=True)

    def __repr__(self):
        return '<Student %r>' % self.name


In [4]:
class Course(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(120), unique=False, nullable=False)
    details = db.Column(db.String(80), unique=False, nullable=False)
    timing = db.Column(db.String(80), unique=False, nullable=False)
    location = db.Column(db.String(80), unique=False, nullable=False)
    requiresTest = db.Column(db.Boolean, nullable=False)
    
    teacher_id = db.Column(db.Integer, db.ForeignKey('teacher.id'), nullable=False)
    teacher = db.relationship('Teacher', backref=db.backref('courses', lazy=True, cascade='all, delete-orphan'))
    student_id = db.Column(db.Integer, db.ForeignKey('student.id'), nullable=True)


In [5]:
class Teacher(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(80), unique=False, nullable=False)
    image = db.Column(db.String(100), nullable=False)
    mobile = db.Column(db.String(120), unique=False, nullable=False)
    email = db.Column(db.String(80), unique=False, nullable=False)



In [6]:
def filetoPath(file):
    if file and file.filename.endswith('.png'):
            filename = file.filename
            save_path = os.path.join('static','assets',)
            if not os.path.exists(save_path):
                os.makedirs(save_path)
            file.save(os.path.join(save_path, filename))
    return filename

Default Route

In [7]:
@app.route('/')
def index():
    if 'course_ids' not in session:
           session.setdefault('course_ids', [])
    return render_template('index.html')



Upload Teacher,student and Course Data to dataBase

In [8]:


@app.route('/admin')
def admin():

    students = Student.query.all()
    teachers = Teacher.query.all()
    courses = Course.query.all()

    return render_template('admin.html', teachers=teachers, courses=courses , students=students)

@app.route('/submit_teacher', methods=['POST'])
def submit_teacher():

    name = request.form['name']
    file = request.files['image']
    image = filetoPath(file)
    mobile = request.form['mobile']
    email = request.form['email']
    
    new_teacher = Teacher(name=name, image=image, mobile=mobile, email=email)
    db.session.add(new_teacher)
    db.session.commit()

    return redirect(url_for('admin'))

@app.route('/submit_course', methods=['POST'])
def submit_course():

    name = request.form['name']
    details = request.form['details']
    timing = request.form['timing']
    location = request.form['location']
    requiresTest = request.form['requiresTest']
    requires_test_bool = bool(strtobool(requiresTest)) if requiresTest else False 
    teacher_id = request.form['teacher_id']

    new_course = Course(name=name, details=details, timing=timing, location=location,requiresTest=requires_test_bool,teacher_id=teacher_id )
    db.session.add(new_course)
    db.session.commit()
    
    return redirect(url_for('admin'))



@app.route('/submit_student', methods=['POST'])
def submit_student():

    name = request.form['name']
    gender = request.form['gender']
    dob = request.form['dob']
    file = request.files['imageInput']
    image = filetoPath(file)
    mobile = request.form['mobile']
    email = request.form['email']
    parent_mobile = request.form['parentmobile']

    new_student = Student(name=name, gender=gender, dob=dob, mobile=mobile, image=image, email=email, parentMobile=parent_mobile)

    for course_id in session['course_ids']:
        course = Course.query.get(course_id)
        if course:
            new_student.courses.append(course)


    db.session.add(new_student)
    db.session.commit()

    return redirect(url_for('index'))

Handle Deleat data form DataBase  admin page and cascade deleation

In [9]:


@app.route('/delete_teacher/<int:teacher_id>', methods=['GET','POST'])
def delete_teacher(teacher_id):
    
    teacher = Teacher.query.get_or_404(teacher_id)
    db.session.delete(teacher)
    db.session.commit()
    
    return redirect(url_for('admin'))

@app.route('/delete_course/<int:course_id>', methods=['GET','POST'])
def delete_course(course_id):
    
    course = Course.query.get_or_404(course_id)
    db.session.delete(course)
    db.session.commit()
    
    return redirect(url_for('admin'))


@app.route('/delete_student/<int:student_id>', methods=['GET','POST'])
def delete_student(student_id):
    
    student = Student.query.get_or_404(student_id)
    db.session.delete(student)
    db.session.commit()
    
    return redirect(url_for('admin'))

Main Page Navigation

In [10]:
@app.route('/courses')
def courses():

    courses = Course.query.all()
    

    return render_template('courses.html',courses=courses)


@app.route('/get_selected_data')
def get_selected_data():
    selected_data = session['course_ids']  # Assuming session['course_ids'] contains your data
    return jsonify(selected_data)


@app.route('/about')
def about():
    return render_template('about.html',)




@app.route('/apply')
def apply():    

    studentCourses=[]
    for course_id in session['course_ids']:

        course = Course.query.get(course_id)
        studentCourses.append(course)
            
    return render_template('apply.html', studentCourses=studentCourses)


@app.route('/enroleCourseId', methods=['POST'])
def enroleCourseId():

    course_id = request.form['course_id']
    
    print(course_id)
    
    if course_id in session['course_ids']:
        session['course_ids'].remove(str(course_id))
    else:
        session['course_ids'].append(str(course_id))
    
    session.modified = True
    print(session['course_ids'])
    return redirect(url_for('apply'))



@app.route('/coursDetail' , methods=['POST'])
def coursDetail():
    
    course_id = request.form.get('course_id')
    course = Course.query.get(course_id)

    if course_id in session['course_ids']:
        enroled = True
    else:
        enroled = False

    return render_template('coursDetail.html', course=course , enroled=enroled )

APP Start

In [11]:

if __name__ == '__main__':
    with app.app_context():
        db.create_all()
    app.run(host='0.0.0.0', debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.106:5000
Press CTRL+C to quit


192.168.0.106 - - [15/Mar/2024 01:23:09] "GET / HTTP/1.1" 200 -
192.168.0.106 - - [15/Mar/2024 01:23:09] "GET /static/js/background.js HTTP/1.1" 304 -
192.168.0.106 - - [15/Mar/2024 01:23:09] "GET /static/css/styles.css HTTP/1.1" 304 -
192.168.0.106 - - [15/Mar/2024 01:23:09] "GET /static/assets/studentStand.png HTTP/1.1" 304 -
192.168.0.106 - - [15/Mar/2024 01:23:09] "GET /static/assets/map.png HTTP/1.1" 304 -
